In [1]:
# Importing required libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer

2022-10-22 18:11:18.075032: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-22 18:11:18.224245: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-22 18:11:18.224269: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-22 18:11:18.253279: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-22 18:11:19.191362: W tensorflow/stream_executor/platform/de

In [2]:
psText = ""
psFileName = "ponniyinSelvan"

import io
for i in range(1,6):
    with io.open(psFileName+str(i)+".txt",'r',encoding='utf8') as f:
        text = f.read()
        psText = psText +text

In [3]:
collectText = ""
collectFileName = "collection"

import io
for i in range(1,11):
    with io.open(collectFileName+str(i)+".txt",'r',encoding='utf8') as f:
        text = f.read()
        collectText = collectText +text

In [5]:
paras = []
import re

paras = re.split('\"|\'|\n|\t|\?|!|\.',psText)

In [6]:
paras1 = []
import re

paras1 = re.split('\"|\'|\n|\t|\?|!|\.',collectText)

In [7]:
sentences = []
for a in paras:
    a = re.sub('^ ','',a)
    a = re.sub(' $','',a)
    a = re.sub(',',' ',a)
    a = re.sub('  ',' ',a)
    if len(a)>1:
        sentences.append(a)

In [8]:
sentences1 = []
for a in paras1:
    a = re.sub('^ ','',a)
    a = re.sub(' $','',a)
    a = re.sub(',',' ',a)
    a = re.sub('  ',' ',a)
    if len(a)>1:
        sentences1.append(a)

In [9]:
import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

In [10]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [11]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [14]:
tokens = []
for sentence in sentences:
    a = list(sentence.split())
    for items in a:
        if len(items) >0:
            tokens.append(items)
for sentence in sentences1:
    a = list(sentence.split())
    for items in a: 
        if len(items) >0:
            tokens.append(items)      

In [16]:
vocab, index = {}, 1  # start indexing from 1
vocab['<pad>'] = 0  # add a padding token
for token in tokens:
    if token not in vocab:
        vocab[token] = index
        index += 1
vocab_size = len(vocab)


In [17]:
inverse_vocab = {index: token for token, index in vocab.items()}


In [18]:
#Vectorize your sentence
vectorSequences = []
for sentence in sentences:
    tokens = list(sentence.split())
    s1 = [vocab[word] for word in tokens]
    vectorSequences.append(s1)
for sentence in sentences1:
    tokens = list(sentence.split())
    s1 = [vocab[word] for word in tokens]
    vectorSequences.append(s1)    

In [26]:
window_size = 2
skipgramList = []
a = vectorSequences[250]
for a in vectorSequences:
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(      a,      vocabulary_size=vocab_size,      window_size=window_size,      negative_samples=0)
    for b in positive_skip_grams:
        skipgramList.append(b)


In [30]:
# Get target and context words for one positive skip-gram.
target_word, context_word = skipgramList[1000]

# Set the number of negative samples per positive context.
num_ns = 4

context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,  # class that should be sampled as 'positive'
    num_true=1,  # each positive skip-gram has 1 positive context class
    num_sampled=num_ns,  # number of negative context words to sample
    unique=True,  # all the negative samples should be unique
    range_max=vocab_size,  # pick index of the samples from [0, vocab_size]
    seed=SEED,  # seed for reproducibility
    name="negative_sampling"  # name of this operation
)
print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])

tf.Tensor([ 3492   118 42595   627], shape=(4,), dtype=int64)
['பயந்தவன்', 'காலப்போக்கில்', 'படிக்கத்', 'எடுப்பித்ததை']


2022-10-23 09:16:25.786608: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-23 09:16:25.787028: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-23 09:16:25.787148: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-10-23 09:16:25.787248: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-10-23 09:16:25.787346: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [31]:
# Add a dimension so you can use concatenation (in the next step).
negative_sampling_candidates = tf.expand_dims(negative_sampling_candidates, 1)

# Concatenate a positive context word with negative sampled words.
context = tf.concat([context_class, negative_sampling_candidates], 0)

# Label the first context word as `1` (positive) followed by `num_ns` `0`s (negative).
label = tf.constant([1] + [0]*num_ns, dtype="int64")

# Reshape the target to shape `(1,)` and context and label to `(num_ns+1,)`.
target = tf.squeeze(target_word)
context = tf.squeeze(context)
label = tf.squeeze(label)

In [32]:
print(f"target_index    : {target}")
print(f"target_word     : {inverse_vocab[target_word]}")
print(f"context_indices : {context}")
print(f"context_words   : {[inverse_vocab[c.numpy()] for c in context]}")
print(f"label           : {label}")

target_index    : 234
target_word     : வழக்கம்
context_indices : [  244  3492   118 42595   627]
context_words   : ['அலைமோதிக்', 'பயந்தவன்', 'காலப்போக்கில்', 'படிக்கத்', 'எடுப்பித்ததை']
label           : [1 0 0 0 0]


In [33]:
print("target  :", target)
print("context :", context)
print("label   :", label)

target  : tf.Tensor(234, shape=(), dtype=int32)
context : tf.Tensor([  244  3492   118 42595   627], shape=(5,), dtype=int64)
label   : tf.Tensor([1 0 0 0 0], shape=(5,), dtype=int64)
